In [1]:
!pip install gluonnlp==0.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp310-cp310-linux_x86_64.whl size=557729 sha256=96daf35ff08bd9d72609d06ed27bfca7461a2e658ec929798c0618dcd8ffec56
  Stored in directory: /root/.cache/pip/wheels/fc/5b/9c/3295bb07f7c5544a96303a48988707816f44a536e8e1413922
Successfully built gluonnlp


In [2]:
# 이는 구글 코랩으로 돌린 버전입니다. 그리고 기본 코드는 SKT Brain의 KoBERT를 그대로 가져왔고, 학습 및 테스트 데이터셋만 따로 준비한 것입니다.
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT
# 참고한 사이트: https://github.com/aeddung/ML-DL/blob/main/NLP/koBert_multiclassification.py
#                https://complexoftaste.tistory.com/2 

!pip install mxnet
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers 
!pip install torch

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/sim

In [3]:
!python --version

Python 3.10.11


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import gluonnlp as nlp

In [5]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# GPU 사용 시
device = torch.device("cuda:0")

In [7]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# 학습용 데이터셋 불러오기
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data.csv') 
df.head()

,Unnamed: 0.1,Unnamed: 0,Title,Field
0,0,0.0,발목 인대,정형외과
1,1,1.0,스마일 라식 2일 전 음주,안과
2,2,2.0,왼쪽 허벅지 안쪽아파요,정형외과
3,3,3.0,실수 잘못을 할때마다 너무 ㅈ고싶어요...,정신건강의학과
4,4,4.0,키분포도 궁금점,소아청소년과


In [ ]:
len(df)

In [24]:
# 데이터 전처리
# 분류하는 진료과를 10개로 줄일 것임
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df.head()

,Title,Field
0,발목 인대,정형외과
1,스마일 라식 2일 전 음주,안과
2,왼쪽 허벅지 안쪽아파요,정형외과
3,실수 잘못을 할때마다 너무 ㅈ고싶어요...,정신건강의학과
4,키분포도 궁금점,소아청소년과


In [25]:
df['Field'].unique()

array(['정형외과', '안과', '정신건강의학과', '소아청소년과', '피부과', '소화기내과', '운동치료', '임신',
       '대장, 항문 외과', '신경과', '암센터', '이비인후과', '순환기내과', '신경외과', '생리, 피임',
       '성형외과', '여성질환', '응급의학과', '내과', '비뇨의학과', '외과', '호흡기내과', '가정의학과',
       '건강상담', '종합병원', '흉부외과', '산부인과', '신장내과', '감염내과', '내분비대사내과',
       '혈액종양내과', '마취통증의학과', '영상의학과', '기타', '출산, 산후조리', '류마티스내과', '재활의학과'],
      dtype=object)

In [26]:
drop_col = ['정신건강의학과', '소아청소년과', '운동치료', '암센터', '응급의학과', '가정의학과', '건강상담', '종합병원', '마취통증의학과', '영상의학과', '기타', '재활의학과','성형외과']
df = df[~df['Field'].isin(drop_col)]

In [27]:
df['Field'].unique()

array(['정형외과', '안과', '피부과', '소화기내과', '임신', '대장, 항문 외과', '신경과', '이비인후과',
       '순환기내과', '신경외과', '생리, 피임', '여성질환', '내과', '비뇨의학과', '외과', '호흡기내과',
       '흉부외과', '산부인과', '신장내과', '감염내과', '내분비대사내과', '혈액종양내과', '출산, 산후조리',
       '류마티스내과'], dtype=object)

In [28]:
len(df)

12608

In [31]:
df['Field'].replace(['순환기내과', '신장내과', '감염내과','호흡기내과','내분비대사내과','혈액종양내과','류마티스내과','소화기내과'], '내과', inplace=True)

In [32]:
df['Field'].unique()

array(['정형외과', '안과', '피부과', '내과', '임신', '대장, 항문 외과', '신경과', '이비인후과',
       '신경외과', '생리, 피임', '여성질환', '비뇨의학과', '외과', '흉부외과', '산부인과',
       '출산, 산후조리'], dtype=object)

In [33]:
df['Field'].replace(['대장, 항문 외과' ], '외과', inplace=True)

In [34]:
df['Field'].replace(['임신','생리, 피임', '여성질환', '출산, 산후조리'], '산부인과', inplace=True)

In [35]:
df['Field'].unique()

array(['정형외과', '안과', '피부과', '내과', '산부인과', '외과', '신경과', '이비인후과', '신경외과',
       '비뇨의학과', '흉부외과'], dtype=object)

In [36]:
df['Field'].replace(['신경외과' ], '신경과', inplace=True)

In [42]:
df['Field'].replace(['비뇨의학과' ], '비뇨기과', inplace=True)

In [43]:
df['Field'].unique()

array(['정형외과', '안과', '피부과', '내과', '산부인과', '외과', '신경과', '이비인후과', '비뇨기과',
       '흉부외과'], dtype=object)

In [39]:
num_classes = df['Field'].nunique()

In [40]:
df.isnull().any().any() #결측치 없음

False

In [41]:
len(df) #최종 데이터 개수

12608

In [44]:
# 이건 안해도 될듯 한데..우선 했음
data1 = df[df['Field'] == '정형외과']
data2 = df[df['Field'] == '안과']
data3 = df[df['Field'] == '피부과']
data4 = df[df['Field'] == '내과']
data5 = df[df['Field'] == '산부인과']
data6 = df[df['Field'] == '외과']
data7 = df[df['Field'] == '신경과']
data8 = df[df['Field'] == '이비인후과']
data9 = df[df['Field'] == '비뇨기과']
data10 = df[df['Field'] == '흉부외과']

new_data = data1.append([data2, data3, data4, data5, data6, data7, data8, data9, data10], sort=False)
new_data = pd.DataFrame(new_data)
new_data.head()

<ipython-input-44-bae2a080919c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = data1.append([data2, data3, data4, data5, data6, data7, data8, data9, data10], sort=False)


,Title,Field
0,발목 인대,정형외과
2,왼쪽 허벅지 안쪽아파요,정형외과
30,척추 측만증 어디로 휘었는지 확인해주세요,정형외과
38,장애등급 있는 75세 이상 할머니 고관절수술 보상받을수 있는...,정형외과
60,척추암 등 디스크가아닌 다른 이유로 디스크증상 시 mri로 나...,정형외과


In [45]:
# class 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(new_data['Field'])
new_data['Field'] = encoder.transform(new_data['Field'])
new_data.head()

,Title,Field
0,발목 인대,7
2,왼쪽 허벅지 안쪽아파요,7
30,척추 측만증 어디로 휘었는지 확인해주세요,7
38,장애등급 있는 75세 이상 할머니 고관절수술 보상받을수 있는...,7
60,척추암 등 디스크가아닌 다른 이유로 디스크증상 시 mri로 나...,7


In [46]:
# 라벨링에 대한 맵핑 테이블
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '내과',
 1: '비뇨기과',
 2: '산부인과',
 3: '신경과',
 4: '안과',
 5: '외과',
 6: '이비인후과',
 7: '정형외과',
 8: '피부과',
 9: '흉부외과'}

In [47]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 10086
test shape is: 2522


In [67]:
# train, test가 데이터프레임 형태인데 모델에 들어가는건 리스트여야함
train = train.values.tolist()
test = test.values.tolist()

In [52]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [49]:
# Setting parameters
# 변경없이 그대로 적용하였음
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [68]:
tok = tokenizer.tokenize

data_train = BERTDataset(train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, vocab, max_len, True, False)

In [69]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [70]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = num_classes, # 변경해준 부분
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [71]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# model = torch.load("/content/drive/MyDrive/model.pt") 저장해둔 모델 불러올 때..

In [72]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [73]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [74]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [75]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    

<ipython-input-75-93370a056c8e>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/158 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.310546398162842 train acc 0.171875
epoch 1 train acc 0.40899296302465027


<ipython-input-75-93370a056c8e>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/40 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7187199519230769


  0%|          | 0/158 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2235325574874878 train acc 0.984375
epoch 5 train acc 0.8654959193870753


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 5 test acc 0.7196814903846154


In [89]:
torch.save(model, "/content/drive/MyDrive/model.pt")

In [90]:
# 테스트 문장 예측
test_sentence = ''
test_label = 6 # 실제 진료과

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['증상', '진료과']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, vocab, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)
  print(mapping[torch.argmax(out).item()])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-90-427d0d828d8e>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[ 5.0807, -1.2293, -1.4839,  0.4627, -1.5567,  0.0595,  1.7077, -1.4333,
         -0.7011, -0.1840]], device='cuda:0', grad_fn=<AddmmBackward0>)
내과
